In [1]:
import requests
from pathlib import Path
import xmltodict
import os
import pandas as pd
import io
import datetime
import pymongo
import json
import gzip
import dns
from os import listdir
from os.path import isfile, join

urlBase='https://tisvcloud.freeway.gov.tw/history/vd/'  # 20190624/cms_value_0000.xml.gz   20190624/cms_value_0014.xml.gz
baseDir='../../../data/'
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client.traffic
trType='vd'
collect = db[trType]

#確認該 url 是否可以下載, 例如 20190230 就不存在, 無法下載; 而且必須是附件形式的方可
def isDownloadable(url):
    """
    Does the url contain a downloadable resource
    """
    h = requests.head(url, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    if content_type is None:
        return False
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    return True


#下載某url檔案後, 放在指定目錄下
def downloadFileFromUrl(url, directory):
    filename = directory+'/'+ url.rsplit('/', 1)[1]
    if not os.path.exists(filename):
        r = requests.get(url, allow_redirects=True)
        open(filename, 'wb').write(r.content)

In [2]:
#抓取 CMS 的某一天所有可能的檔案 並下載到指定目錄
def downloadVD5MinDay(trType, day, baseDir):
    downloads=[]
    for hour in range(0,24):
        for mininute in range(0,60,5):  #vd 每5分鐘一次
            downloads.append(urlBase + day + '/' + trType + '_value5_'+ format(hour, '02d')+ format(mininute, '02d') +'.xml.gz' )
    p=baseDir+trType+'/'+day
    Path(p).mkdir(parents=True, exist_ok=True)
    for url in downloads:
        if isDownloadable(url):
            downloadFileFromUrl(url,p)
            #print(url)  

In [14]:
from pandas import json_normalize #package for flattening json in pandas df

def splitLane2Tbl(lanes):
    return json_normalize(data=lanes, record_path='cars', meta=['@vsrid', '@speed','@laneoccupy'])
 

In [25]:
          
def getAllFiles2Mongo(trType, day, baseDir):
    directory=baseDir+trType+'/'+day+'/'
    #print(directory)
    files = [f for f in listdir(directory) if isfile(join(directory, f))]
    
    for file in files:
        print(file)
        f = gzip.open(directory+file, 'rb')
        doc = xmltodict.parse(f)
        Y=doc['XML_Head']['Infos']
        tmp=[]
        if Y is not None:
            X=Y['Info']
            #has datacollecttime, no need updateTime=datetime.datetime.strptime(doc['XML_Head']['@updatetime'], "%Y/%m/%d %H:%M:%S")
            json_data = json.dumps(X)
            cc = json.loads(json_data)
            
            for a in cc:
                tmp.append({"vdid":a['@vdid'], "collecttime": a['@datacollecttime'], 'lanes':splitLane2Tbl(a['lane'])})
            
        else:
            print('TypeError: ',Y)
        
        #collect.insert_many(tmp)
        f.close()
        ######3##########TODO 
        break
    print(tmp[0],'\n',tmp[1],'\n',tmp[2],'\n',)

In [26]:
def downloadInsert(trType, day, baseDir):
    downloadVD5MinDay(trType, day, baseDir)
    getAllFiles2Mongo(trType, day, baseDir)

In [27]:
getAllFiles2Mongo('vd','20190404',baseDir)

vd_value5_0715.xml.gz
{'vdid': 'nfbVD-N1-S--0.008-M-LOOP', 'collecttime': '2019/04/04 07:15:00', 'lanes':   @carid @volume @vsrid @speed @laneoccupy
0      S      33      1     67           7
1      T       4      1     67           7
2      L       0      1     67           7
3      S      39      2     60           8
4      T       1      2     60           8
5      L       0      2     60           8} 
 {'vdid': 'nfbVD-N1-S-0.990-N-LOOP', 'collecttime': '2019/04/04 07:15:00', 'lanes':   @carid @volume @vsrid @speed @laneoccupy
0      S      41      1     95           6
1      T       0      1     95           6
2      L       3      1     95           6
3      S      23      2     91           3
4      T       2      2     91           3
5      L       0      2     91           3} 
 {'vdid': 'nfbVD-N1-S-1-O-NW-1-基隆', 'collecttime': '2019/04/04 07:15:00', 'lanes':   @carid @volume @vsrid @speed @laneoccupy
0      S      20      1     57           3
1      T       0      1     57     

In [ ]:
days=['0405','0406','0407','0606','0607','0608','0624','0625','0626','0627','0628','0202','0203','0204','0205','0206','0207','0208','0209','0210']#
#days=['1010','1011','1012','1013','0913','0914','0915']
for d in days:
    downloadVDDay('vd','2019'+d,baseDir)